In [8]:
###############Query Builder for the FDA Api
import requests
##Base query: https://api.fda.gov/drug/event.json

##query_url = "https://api.fda.gov/drug/event.json?search=receivedate:[20040101+TO+20231207]&count=patient.drug.openfda.pharm_class_epc.exact"

query_url = "https://api.fda.gov/drug/event.json?search=receivedate:[20220101+TO+20231207]&count=patient.drug.medicinalproduct.exact"

res = requests.get(query_url).json()

####Get a full list of all the drug names first. We can use them to look for adverse effects later

names_list = []

for item in range(0,len(res["results"])):
    names_list.append(res["results"][item]["term"])

print(names_list)

####Lovely!



['HUMIRA', 'ENBREL', 'ZANTAC', 'REVLIMID', 'METFORMIN', 'ASPIRIN', 'PREDNISONE.', 'AMLODIPINE', 'METHOTREXATE', 'ATORVASTATIN', 'ASPIRIN.', 'LYRICA', 'DUPIXENT', 'OMEPRAZOLE.', 'PROACTIV MD ADAPALENE ACNE TREATMENT', 'TYSABRI', 'PREDNISONE', 'LIPITOR', 'OXYCONTIN', 'XARELTO', 'NEXIUM', 'SYNTHROID', 'ELIQUIS', 'GABAPENTIN.', 'PANTOPRAZOLE', 'REMICADE', 'METHOTREXATE.', 'LISINOPRIL.', 'VITAMIN D', 'DEXAMETHASONE.', 'PROACTIV MD DEEP CLEANSING FACE WASH', 'AVONEX', 'PROLIA', 'PROACTIV MD DAILY OIL CONTROL SPF 30', 'COSENTYX', 'LASIX', 'OTEZLA', 'FUROSEMIDE.', 'VITAMIN D3', 'CALCIUM', 'RANITIDINE.', 'TYLENOL', 'MIRENA', 'SPIRIVA', 'OMEPRAZOLE', 'TECFIDERA', 'LANTUS', 'CYMBALTA', 'REPATHA', 'CRESTOR', 'LISINOPRIL', 'FOLIC ACID.', 'ACETAMINOPHEN', 'PLAVIX', 'FORTEO', 'SIMVASTATIN.', 'ENTRESTO', 'PRILOSEC', 'FOLIC ACID', 'HUMALOG', 'AVANDIA', 'IBUPROFEN.', 'DEXAMETHASONE', 'VITAMIN B12', 'CELEBREX', 'LEVOTHYROXINE.', 'XELJANZ', 'ORENCIA', 'PARACETAMOL', 'METOPROLOL.', 'FUROSEMIDE', 'XANAX', '

In [43]:
########Now we get the concrete calls:
import pandas as pd 

second_query = 'https://api.fda.gov/drug/event.json?search=(receivedate:[20040101+TO+20231207])+AND+patient.drug.medicinalproduct:"ASPIRIN"&limit=50'

res_02 = requests.get(second_query).json()

#print(res_02["results"][0])

#Trial DF

dataframe_setup = []

for patient in range(0,len(res_02["results"])):
    #print(patient)
    ##Create a new line
    line = []

    ###Simplified: Serious, weight, sex, date, reactions get appended inndeiately without specification
    reactions = []
    for reaction in range(0,len(res_02["results"][patient]["patient"]['reaction'])):
        reactions.append(res_02["results"][patient]["patient"]['reaction'][reaction]['reactionmeddrapt'])
        #print(res_02["results"][patient]["patient"]['reaction'][reaction]['reactionmeddrapt'])

    try:
        line.append(res_02["results"][patient]["serious"])
    except:
        line.append("N/A")
    try:
        line.append(res_02["results"][patient]["patient"]["patientweight"])
    except: 
        line.append("N/A")
    try:
        line.append(res_02["results"][patient]["patient"]["patientsex"])
    except:
        line.append("N/A")
    try:    
        line.append(res_02["results"][patient]["receivedate"])
    except:
        line.append("N/A")
    try:
        line.append(reactions)
    except:
        line.append("N/A")

    #print(line)

    ###Append Line to dataframe_setup
    dataframe_setup.append(line)


#print(dataframe_setup)
#######Create Dataframe:

patients = pd.DataFrame(dataframe_setup, columns =['Serious', 'Weight', "sex", "date", "reactions"]) 

print(patients)



   Serious  Weight  sex      date  \
0        2     N/A    2  20140312   
1        2     N/A    2  20140312   
2        1     N/A    2  20140312   
3        1     N/A    2  20140312   
4        2    82.1    1  20140312   
5        1     N/A    1  20140312   
6        1     N/A    1  20140312   
7        2     N/A    2  20140312   
8        1   74.83    2  20140312   
9        1   65.31    1  20140312   
10       1   83.54    1  20140312   
11       1   92.06    1  20140312   
12       1   65.31    2  20140312   
13       1   62.13    2  20140312   
14       2     N/A    1  20140312   
15       2     N/A  N/A  20140312   
16       1     N/A    0  20140312   
17       2     N/A    1  20140312   
18       2     N/A    2  20140312   
19       2     N/A    2  20140312   
20       2   72.56    2  20140312   
21       2     N/A    1  20140312   
22       2     N/A    2  20140312   
23       2    56.2    2  20140312   
24       1      98    1  20140312   
25       1     N/A    1  20140312   
2

In [36]:
###Easier: Find cases based on symptoms, then suggest drugs


second_query = 'https://api.fda.gov/drug/event.json?search=(receivedate:[20040101+TO+20231207])+AND+patient.patientsex:1+AND+patient.drug.medicinalproduct:"ASPIRIN"&limit=5'

res_02 = requests.get(second_query).json()

print(res_02["results"][1])

{'safetyreportversion': '1', 'safetyreportid': '10003595', 'primarysourcecountry': 'US', 'occurcountry': 'US', 'transmissiondateformat': '102', 'transmissiondate': '20141002', 'reporttype': '1', 'serious': '1', 'seriousnesshospitalization': '1', 'receivedateformat': '102', 'receivedate': '20140312', 'receiptdateformat': '102', 'receiptdate': '20140312', 'fulfillexpeditecriteria': '2', 'companynumb': 'US-INCYTE CORPORATION-2014IN000070', 'duplicate': '1', 'reportduplicate': {'duplicatesource': 'INCYTE', 'duplicatenumb': 'US-INCYTE CORPORATION-2014IN000070'}, 'primarysource': {'reportercountry': 'US', 'qualification': '1'}, 'sender': {'sendertype': '2', 'senderorganization': 'FDA-Public Use'}, 'receiver': {'receivertype': '6', 'receiverorganization': 'FDA'}, 'patient': {'patientsex': '1', 'reaction': [{'reactionmeddraversionpt': '17.0', 'reactionmeddrapt': 'Transfusion', 'reactionoutcome': '6'}], 'drug': [{'drugcharacterization': '1', 'medicinalproduct': 'JAKAFI', 'drugstructuredosagenum

Here is an environment in which you can try stuff out